In [1]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import matplotlib.dates as mdates
from datetime import datetime, date

# preamble for jupyter notebook and django
import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/muef/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
    dtm_path = "/home/galm/software/dtm/dtm/main"
else:
    # local paths
    sys.path.append('/media/Data/MCC/tmv/BasicBrowser/')
    dtm_path = "/home/finn/dtm/dtm/main"

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
import scoping.models as sm
from parliament.tasks import do_search, run_tm, combine_searches
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import * 
from utils.tm_mgmt import update_topic_scores
from django.db.models import Q, Count, Func, F, Sum, Avg, Value as V
from django.db.models.functions import TruncDate, TruncMonth, TruncYear

In [23]:
# topic labeling

run_id = 2111
no_top_words = 20

labels = {}

df = pd.DataFrame(columns=['label', 'top words', 'score pct', 'score'])

get_input = False

for topic in Topic.objects.filter(run_id = run_id):
    stat = RunStats.objects.get(run_id=run_id)

    print("Topic #{}".format(topic.id))
    
    if stat.method == 'BD':
        
        terms = Term.objects.filter(topicterm__topic=topic).values('title').annotate(
    summed_score = Sum(F('topicterm__score'))).order_by('-summed_score')[:20]
        term_list = [t['title'] for t in terms]
        print(", ".join(term_list))

    else:
        print("Top words: {}".format(topic.top_words))

    if get_input:
        label = input("topic label: ")
        if label:
            labels[topic.id] = label
            topic.title = label
            topic.save()
    else:
        print("topic label: {}".format(topic.title))
    
    print("")
    
    df.loc[topic.id] = {"top words": ", ".join(term_list), "score": topic.score, "label": topic.title, "score pct": 0}

print(labels)
df['score pct'] = 100 * df['score'] / df['score'].sum()
df = df.sort_values(by=['score'], ascending=False)

Topic #177279
gesetz, gesetzentwurf, entwurf, zustimm, bundesrat, zweit, dritt, punkt, wort, erheb, einstimm, lesung, anderungsantrag, art, bundestag, begrund, haus, fall, beschloss, vorlieg
topic label: {gesetz, gesetzentwurf, entwurf}

Topic #177278
kernenergi, kernkraftwerk, ausstieg, atomenergi, nutzung, friedlich, radioaktiv, gefahr, anlag, bau, atomkraftwerk, bundesregier, entsorg, betrieb, bevolker, deutschland, atomgesetz, tschernobyl, kons, reaktor
topic label: {kernenergi, kernkraftwerk, ausstieg}

Topic #177277
zwischenfrag, deutschland, ziel, energiew, letzt, prozent, strom, gestatt, dank, energi, erneuerbar, wettbewerbsfah, grun, geehrt, zukunft, gerad, verlass, arbeitsplatz, bereich, beim
topic label: {zwischenfrag, deutschland, ziel}

Topic #177276
dank, grun, kolleginn, endlich, bundesregier, einfach, deutschland, geehrt, wirklich, eigent, regier, minist, leid, problem, verantwort, sorg, statt, kohleausstieg, find, kommt
topic label: {dank, grun, kolleginn}

Topic #1772

bundesregier, hoh, staatssekretar, wirtschaft, grund, insbesond, deutsch, rahm, entsprech, fall, bundesminist, dah, beim, allerding, erheb, vorgeseh, zusammenhang, erford, auffass, seit
topic label: {bundesregier, hoh, staatssekretar}

Topic #177240
kilowattstund, energiew, netz, rwe, strom, erneuerbar, energi, ausbau, kohlekraftwerk, kost, bau, konzern, windenergi, energiekonzern, burg, zusatz, cent, zahl, netzausbau, stell
topic label: {kilowattstund, energiew, netz}

Topic #177239
deutsch, ausland, volk, amerikan, wirtschaft, regier, interess, hoh, arbeit, westdeutschland, bundeskanzl, krieg, kampf, bevolker, industri, westdeutsch, seit, adenau, demokrat, deutschland
topic label: {deutsch, ausland, volk}

Topic #177238
russland, russisch, ukrain, ccs, dank, grun, mal, deutschland, strukturwandel, krisch, kolleginn, koalitionsvertrag, red, gas, natur, desweg, prozent, kohleausstieg, abhang, ccstechnologi
topic label: {russland, russisch, ukrain}

Topic #177237
kommission, braunkohl, 

In [31]:
df

,label,top words,score pct,score
177264,"{darauf, darub, natur}","darauf, darub, natur, richtig, stell, wiss, tu...",23.165054,939731.269529
177261,"{regier, red, bundeskanzl}","regier, red, bundeskanzl, wirklich, sei, sagt,...",7.055709,286227.286681
177274,"{wirtschaft, bundesregier, sozial}","wirtschaft, bundesregier, sozial, deutsch, ges...",5.460566,221517.472968
177241,"{bundesregier, hoh, staatssekretar}","bundesregier, hoh, staatssekretar, wirtschaft,...",4.228513,171537.082003
177251,"{milliard, bund, rund}","milliard, bund, rund, million, hoh, ausgab, be...",3.476431,141027.574810
177250,"{wirtschaft, wachstum, entwickl}","wirtschaft, wachstum, entwickl, investition, o...",3.311777,134348.092923
177242,"{europa, gemeinschaft, union}","europa, gemeinschaft, union, vertrag, national...",3.212921,130337.805275
177269,"{gesetz, wirtschaft, bundesregier}","gesetz, wirtschaft, bundesregier, bundesminist...",2.764812,112159.465356
177244,"{energi, strom, gas}","energi, strom, gas, erneuerbar, regenerativ, w...",2.737208,111039.675321
177249,"{million, tonn, kohl}","million, tonn, kohl, preis, bundeswirtschaftsm...",2.682145,108805.956757


In [33]:
fname = "./plots/topic_analysis_{}/0_topic_list.xlsx".format(run_id)
writer = pd.ExcelWriter(fname, engine='xlsxwriter')

df.to_excel(writer)

In [13]:
# modify single labels:

topic = Topic.objects.get(id=167080)
topic.title = 'mixed 3'
topic.save()